# Setup S3 Environment
Import required libraries and set up logging configurations. Configure AWS credentials and region for testing.

In [ ]:
# Import required libraries
import logging
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

# Set up logging configurations
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Configure AWS credentials and region for testing
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
aws_region = 'YOUR_AWS_REGION'

# Initialize the S3 client
try:
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=aws_region
    )
    logger.info("S3 client initialized successfully.")
except (NoCredentialsError, PartialCredentialsError) as e:
    logger.error("AWS credentials not found or incomplete. Please check your configuration.")
    raise e

# Initialize S3 Client and Managers
Create S3Client instance and initialize BucketManager and ObjectManager with debug output for connection verification.

In [ ]:
# Import required libraries for S3 operations
from s3_operations.s3_client import S3Client
from s3_operations.bucket_manager import BucketManager
from s3_operations.object_manager import ObjectManager

# Initialize S3Client instance
s3_client_instance = S3Client().get_client()

# Initialize BucketManager and ObjectManager with the S3 client
bucket_manager = BucketManager(s3_client_instance)
object_manager = ObjectManager(s3_client_instance)

# Debug output to verify connection
try:
    # List buckets to verify connection
    buckets = s3_client_instance.list_buckets()
    logger.info("S3 connection verified. Buckets available: %s", [bucket['Name'] for bucket in buckets['Buckets']])
except Exception as e:
    logger.error("Failed to verify S3 connection: %s", e)
    raise e

# Bucket Operations
Test bucket creation with unique names. Add debugging statements to verify bucket existence and creation status.

In [ ]:
from datetime import datetime

# Function to create a unique bucket name
def create_unique_bucket_name(base_name):
    return f"{base_name}-{datetime.now().strftime('%Y%m%d%H%M%S')}"

# Create a unique bucket name
test_bucket_name = create_unique_bucket_name("test-bucket")
logger.info("Generated unique bucket name: %s", test_bucket_name)

# Attempt to create the bucket
bucket_created = bucket_manager.create_bucket(test_bucket_name)
if bucket_created:
    logger.info("Bucket '%s' created successfully.", test_bucket_name)
else:
    logger.error("Failed to create bucket '%s'.", test_bucket_name)

# Verify bucket existence
bucket_exists = bucket_manager.check_bucket_exists(test_bucket_name)
if bucket_exists:
    logger.info("Bucket '%s' exists.", test_bucket_name)
else:
    logger.error("Bucket '%s' does not exist.", test_bucket_name)

# Object Upload and Download
Create test files, upload them to S3, and verify the upload process. Include progress tracking and file integrity checks.

In [ ]:
# Object Upload and Download

import tempfile
import os

# Create a temporary test file
with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
    tmp_file.write(b"Test content for S3 upload")
    tmp_path = tmp_file.name

logger.info("Temporary test file created at: %s", tmp_path)

# Upload the test file to the S3 bucket
upload_success = object_manager.upload_file(test_bucket_name, tmp_path, "test-file.txt")
if upload_success:
    logger.info("File uploaded successfully to bucket '%s' as 'test-file.txt'.", test_bucket_name)
else:
    logger.error("Failed to upload file to bucket '%s'.", test_bucket_name)

# List objects in the bucket to verify upload
logger.info("Listing objects in bucket '%s':", test_bucket_name)
for page in object_manager.list_objects_paginated(test_bucket_name):
    for obj in page:
        logger.info(" - %s", obj)

# Download the uploaded file
download_path = "/tmp/downloaded_test_file.txt"
download_success = object_manager.download_file(test_bucket_name, "test-file.txt", download_path)
if download_success:
    logger.info("File downloaded successfully from bucket '%s' to '%s'.", test_bucket_name, download_path)
else:
    logger.error("Failed to download file from bucket '%s'.", test_bucket_name)

# Verify file integrity by comparing content
with open(download_path, 'rb') as downloaded_file:
    downloaded_content = downloaded_file.read()

with open(tmp_path, 'rb') as original_file:
    original_content = original_file.read()

if downloaded_content == original_content:
    logger.info("File integrity check passed. The downloaded file matches the original file.")
else:
    logger.error("File integrity check failed. The downloaded file does not match the original file.")

# Clean up temporary files
os.remove(tmp_path)
os.remove(download_path)
logger.info("Temporary files cleaned up.")

# List and Paginate Objects
Demonstrate object listing with pagination, including detailed inspection of object metadata and debug outputs.

In [ ]:
# List and Paginate Objects

# List objects in the bucket with pagination and detailed inspection
logger.info("Listing objects in bucket '%s' with pagination and detailed inspection:", test_bucket_name)
for page in object_manager.list_objects_paginated(test_bucket_name):
    for obj in page:
        logger.info("Object Key: %s", obj['Key'])
        logger.info(" - Last Modified: %s", obj['LastModified'])
        logger.info(" - Size: %d", obj['Size'])
        logger.info(" - Storage Class: %s", obj['StorageClass'])
        logger.info(" - ETag: %s", obj['ETag'])

# Error Handling and Debugging
Test various error scenarios and add exception handling with detailed error messages and stack traces.

In [ ]:
# Error Handling and Debugging

import traceback

# Test error scenario: Attempt to create a bucket with an invalid name
invalid_bucket_name = "Invalid_Bucket_Name_With_Uppercase"
try:
    bucket_manager.create_bucket(invalid_bucket_name)
except Exception as e:
    logger.error("Error creating bucket with invalid name '%s': %s", invalid_bucket_name, e)
    logger.debug("Stack trace:\n%s", traceback.format_exc())

# Test error scenario: Attempt to upload a file to a non-existent bucket
non_existent_bucket = "non-existent-bucket"
try:
    object_manager.upload_file(non_existent_bucket, tmp_path, "test-file.txt")
except Exception as e:
    logger.error("Error uploading file to non-existent bucket '%s': %s", non_existent_bucket, e)
    logger.debug("Stack trace:\n%s", traceback.format_exc())

# Test error scenario: Attempt to download a non-existent file
try:
    object_manager.download_file(test_bucket_name, "non-existent-file.txt", download_path)
except Exception as e:
    logger.error("Error downloading non-existent file from bucket '%s': %s", test_bucket_name, e)
    logger.debug("Stack trace:\n%s", traceback.format_exc())

# Test error scenario: Attempt to delete a non-existent bucket
try:
    bucket_manager.delete_bucket(non_existent_bucket)
except Exception as e:
    logger.error("Error deleting non-existent bucket '%s': %s", non_existent_bucket, e)
    logger.debug("Stack trace:\n%s", traceback.format_exc())

# Cleanup Operations
Implement and verify cleanup procedures for test files and buckets with confirmation checks.

In [ ]:
# Cleanup Operations

# Delete the uploaded test file from the S3 bucket
delete_success = object_manager.delete_file(test_bucket_name, "test-file.txt")
if delete_success:
    logger.info("File 'test-file.txt' deleted successfully from bucket '%s'.", test_bucket_name)
else:
    logger.error("Failed to delete file 'test-file.txt' from bucket '%s'.", test_bucket_name)

# Verify the file deletion by listing objects again
logger.info("Verifying file deletion by listing objects in bucket '%s':", test_bucket_name)
for page in object_manager.list_objects_paginated(test_bucket_name):
    for obj in page:
        logger.info(" - %s", obj)

# Delete the test bucket
bucket_delete_success = bucket_manager.delete_bucket(test_bucket_name)
if bucket_delete_success:
    logger.info("Bucket '%s' deleted successfully.", test_bucket_name)
else:
    logger.error("Failed to delete bucket '%s'.", test_bucket_name)

# Verify the bucket deletion
bucket_exists = bucket_manager.check_bucket_exists(test_bucket_name)
if not bucket_exists:
    logger.info("Bucket '%s' confirmed deleted.", test_bucket_name)
else:
    logger.error("Bucket '%s' still exists after deletion attempt.", test_bucket_name)